# Feed Forward Neural Network - Diabetes

### Imports

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset, DataLoader

c:\Users\msoum\anaconda3\envs\pytorch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Load Dataset

In [2]:
url = "https://raw.githubusercontent.com/fawazsammani/The-Complete-Neural-Networks-Bootcamp-Theory-Applications/master/diabetes.csv"

In [3]:
df = pd.read_csv(url)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 8 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Number of times pregnant      768 non-null    int64  
 1   Plasma glucose concentration  768 non-null    int64  
 2   Diastolic blood pressure      768 non-null    int64  
 3   Triceps skin fold thickness   768 non-null    int64  
 4   2-Hour serum insulin          768 non-null    int64  
 5   Body mass index               768 non-null    float64
 6   Age                           768 non-null    int64  
 7   Class                         768 non-null    object 
dtypes: float64(1), int64(6), object(1)
memory usage: 48.1+ KB


In [5]:
df.head()

,Number of times pregnant,Plasma glucose concentration,Diastolic blood pressure,Triceps skin fold thickness,2-Hour serum insulin,Body mass index,Age,Class
0,6,148,72,35,0,33.6,50,positive
1,1,85,66,29,0,26.6,31,negative
2,8,183,64,0,0,23.3,32,positive
3,1,89,66,23,94,28.1,21,negative
4,0,137,40,35,168,43.1,33,positive


### Data Preprocessing

In [6]:
x = df.drop("Class", axis=1).values

In [7]:
y = df["Class"].values

In [8]:
x[:5]

array([[  6. , 148. ,  72. ,  35. ,   0. ,  33.6,  50. ],
       [  1. ,  85. ,  66. ,  29. ,   0. ,  26.6,  31. ],
       [  8. , 183. ,  64. ,   0. ,   0. ,  23.3,  32. ],
       [  1. ,  89. ,  66. ,  23. ,  94. ,  28.1,  21. ],
       [  0. , 137. ,  40. ,  35. , 168. ,  43.1,  33. ]])

In [9]:
x.shape

(768, 7)

In [10]:
y[:5]

array(['positive', 'negative', 'positive', 'negative', 'positive'],
      dtype=object)

In [11]:
# Replace "positive" with 1 and "negative" with 0
y = np.where(y == "positive", 1, 0).astype(np.float64)

In [12]:
y[:5]

array([1., 0., 1., 0., 1.])

### Data Normalization

In [13]:
standard_scaler = StandardScaler()

In [14]:
x = standard_scaler.fit_transform(x)

In [15]:
x[:5]

array([[ 0.63994726,  0.84832379,  0.14964075,  0.90726993, -0.69289057,
         0.20401277,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575,  0.53090156, -0.69289057,
        -0.68442195, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, -1.28821221, -0.69289057,
        -1.10325546, -0.10558415],
       [-0.84488505, -0.99820778, -0.16054575,  0.15453319,  0.12330164,
        -0.49404308, -1.04154944],
       [-1.14185152,  0.5040552 , -1.50468724,  0.90726993,  0.76583594,
         1.4097456 , -0.0204964 ]])

### Convert to PyTorch Tensor

In [16]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [17]:
x = torch.tensor(x)
y = torch.tensor(y).unsqueeze(1)

In [18]:
print(x.shape)
print(y.shape)

torch.Size([768, 7])
torch.Size([768, 1])


### Custom PyTorch Dataset

In [19]:
class DiabetesDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

In [20]:
dataset = DiabetesDataset(x, y)

In [21]:
len(dataset)

768

In [22]:
dataset[0]

(tensor([ 0.6399,  0.8483,  0.1496,  0.9073, -0.6929,  0.2040,  1.4260],
        dtype=torch.float64),
 tensor([1.], dtype=torch.float64))

### Data Loader

In [23]:
train_loader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=0)

In [24]:
print(f"There are {len(train_loader)} batches in the train_loader")
for x, y in train_loader:
    print(f"For each batch, data is {x.shape} and labels are {y.shape}")
    break

There are 24 batches in the train_loader
For each batch, data is torch.Size([32, 7]) and labels are torch.Size([32, 1])


### Neural Network

In [25]:
class Model(nn.Module):
    def __init__(self, input_features, output_features):
        super().__init__()
        self.fc1 = nn.Linear(input_features, 5)
        self.fc2 = nn.Linear(5, 4)
        self.fc3 = nn.Linear(4, 3)
        self.fc4 = nn.Linear(3, output_features)
        self.tanh = nn.Tanh()
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        out = self.tanh(self.fc1(x))
        out = self.tanh(self.fc2(out))
        out = self.tanh(self.fc3(out))
        out = self.sigmoid(self.fc4(out))
        return out

In [26]:
model = Model(x.shape[1], 1).to(device)

### Criterion & Optimizer

In [27]:
criterion = nn.BCELoss(reduction="mean") # default reduction is "mean"

In [28]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)

### Training

In [29]:
epochs = 200

for epoch in range(epochs):
    for inputs, labels in train_loader:
        inputs = inputs.float().to(device)
        labels = labels.float().to(device)
        # Forward pass
        predictions = model(inputs) # Equivalent to model.forward(x)
        # Compute loss
        loss = criterion(predictions, labels)
        # Backward pass
        optimizer.zero_grad() # Clear gradients
        loss.backward() # Compute gradients
        optimizer.step() # Update parameters
        
    # Accuracy
    with torch.no_grad():
        if (epoch + 1) % 10 == 0:
            accuracy = (predictions.round() == labels).float().mean()
            print(f"Epoch {epoch} - Loss: {loss:.4f} - Accuracy: {accuracy:.4f}")

Epoch 0 - Loss: 0.6148 - Accuracy: 0.6562
Epoch 10 - Loss: 0.3275 - Accuracy: 0.9062
Epoch 20 - Loss: 0.5738 - Accuracy: 0.7812
Epoch 30 - Loss: 0.3709 - Accuracy: 0.8125
Epoch 40 - Loss: 0.4585 - Accuracy: 0.8125
Epoch 50 - Loss: 0.3240 - Accuracy: 0.8438
Epoch 60 - Loss: 0.4724 - Accuracy: 0.7500
Epoch 70 - Loss: 0.5512 - Accuracy: 0.7500
Epoch 80 - Loss: 0.4170 - Accuracy: 0.7500
Epoch 90 - Loss: 0.2579 - Accuracy: 0.8438
Epoch 100 - Loss: 0.4185 - Accuracy: 0.8125
Epoch 110 - Loss: 0.4361 - Accuracy: 0.8438
Epoch 120 - Loss: 0.4581 - Accuracy: 0.6875
Epoch 130 - Loss: 0.3866 - Accuracy: 0.8438
Epoch 140 - Loss: 0.5244 - Accuracy: 0.7188
Epoch 150 - Loss: 0.3343 - Accuracy: 0.8438
Epoch 160 - Loss: 0.3523 - Accuracy: 0.8438
Epoch 170 - Loss: 0.5496 - Accuracy: 0.6562
Epoch 180 - Loss: 0.3875 - Accuracy: 0.7812
Epoch 190 - Loss: 0.5191 - Accuracy: 0.7500
